In [1]:
import tensorflow as tf
keras = tf.keras

c:\users\aakaas~1\virtua~1\ufacgi~1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
Input = keras.layers.Input
BatchNormalization = keras.layers.BatchNormalization
Conv2D = keras.layers.Conv2D
UpSampling2D = keras.layers.UpSampling2D
Add = keras.layers.Add
Model = keras.models.Model

In [3]:
input_layer = Input(shape=(224, 224, 3))

In [4]:
vgg16 = keras.applications.vgg16.VGG16(include_top=False,
                                       weights='imagenet',
                                       input_shape=(224, 224, 3),
                                       pooling=None, 
                                       input_tensor=input_layer)
layers = {layer.name: layer for layer in vgg16.layers}

In [5]:
block5_bn = BatchNormalization()(layers['block5_pool'].output)
block5_bn_conv = Conv2D(filters=512, kernel_size=1, activation='relu', padding='same')(block5_bn)
block5_up = UpSampling2D(size=(2, 2))(block5_bn_conv)

block4_bn = BatchNormalization()(layers['block4_pool'].output)
block45_add = Add()([block4_bn, block5_up])
block45_add_conv = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(block45_add)
block45_up = UpSampling2D(size=(2, 2))(block45_add_conv)

block3_bn = BatchNormalization()(layers['block3_pool'].output)
block34_add = Add()([block3_bn, block45_up])
block34_add_conv = Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(block34_add)
block34_up = UpSampling2D(size=(2, 2))(block34_add_conv)

block2_bn = BatchNormalization()(layers['block2_pool'].output)
block23_add = Add()([block2_bn, block34_up])
block23_add_conv = Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(block23_add)
block23_up = UpSampling2D(size=(2, 2))(block23_add_conv)

block1_bn = BatchNormalization()(layers['block1_pool'].output)
block12_add = Add()([block1_bn, block23_up])
block12_add_conv = Conv2D(filters=3, kernel_size=3, activation='relu', padding='same')(block12_add)

block12_up = UpSampling2D(size=(2, 2))(block12_add_conv)
output = Conv2D(filters=3, kernel_size=3, activation='sigmoid', padding='same')(block12_up)

In [17]:
generator = Model(inputs=input_layer, outputs=output)
all_layers = {layer.name: layer for layer in generator.layers}
for name, layer in layers.items():
    all_layers[name].trainable = False